In [ ]:
#!pip install -U huggingface_hub scikit-learn pandas numpy --quiet


In [ ]:
import os
from huggingface_hub import InferenceClient

from google.colab import userdata


os.environ["HUGGINGFACEHUB_API_TOKEN"] = userdata.get('HUGGINGFACEHUB_API_TOKEN')
os.environ["HUGGINGFACEHUB_API_TOKEN"] = HUGGINGFACEHUB_API_TOKEN

# Use LLaMA-3 or LLaMA-2 chat model
client = InferenceClient(
    model="meta-llama/Meta-Llama-3-8B-Instruct",
    token=HUGGINGFACEHUB_API_TOKEN
)


In [ ]:
import pandas as pd
import numpy as np
from sklearn.ensemble import IsolationForest

def generate_metrics():
    df = pd.DataFrame({
        "cpu": np.random.normal(50, 10, 100),
        "memory": np.random.normal(60, 15, 100),
        "disk_io": np.random.normal(100, 25, 100),
    })
    df.iloc[95:100] += np.random.uniform(100, 200)
    return df

def detect_anomalies(df):
    model = IsolationForest(contamination=0.05, random_state=42)
    model.fit(df)
    df["anomaly"] = model.predict(df)
    return df[df["anomaly"] == -1]


In [ ]:
def build_messages(anomalies_df):
    return [
        {"role": "system", "content": "You are an expert in AIOps."},
        {"role": "user", "content": f"""Detected anomalies (via Isolation Forest):

{anomalies_df.to_string(index=False)}

Please explain the root cause, possible impact, and recommended resolution."""}
    ]


In [ ]:
import time

def evaluate_response(response_text, start_time):
    duration = round(time.time() - start_time, 2)
    words = len(response_text.split())

    keywords = ["cpu", "memory", "disk", "fix", "restart", "resource", "usage"]
    relevance_hits = sum(kw in response_text.lower() for kw in keywords)
    relevance_score = round(relevance_hits / len(keywords), 2)

    return {
        "latency_seconds": duration,
        "word_count": words,
        "relevance_score": relevance_score,
    }


In [ ]:
df = generate_metrics()
anomalies = detect_anomalies(df)

if anomalies.empty:
    print("✅ No anomalies detected.")
else:
    print("🚨 Anomalies detected. Sending to LLaMA...\n")
    messages = build_messages(anomalies)

    start = time.time()
    response = client.chat_completion(messages=messages)
    reply = response.choices[0].message.content.strip()

    print("LLaMA Response:\n", reply)

    metrics = evaluate_response(reply, start)
    print("\n📊 Evaluation Metrics:")
    for k, v in metrics.items():
        print(f" - {k}: {v}")


🚨 Anomalies detected. Sending to LLaMA...

LLaMA Response:
 **Anomaly Detection Analysis**

Based on the provided data, it appears that the anomalies were detected using the Isolation Forest algorithm, which is a robust and efficient method for outlier detection. The data points are characterized by CPU, memory, and disk I/O usage metrics.

**Root Cause Analysis**

After analyzing the data, it appears that the system is experiencing high disk I/O usage, which is likely the primary root cause of the anomalies. The disk I/O usage metrics are significantly higher than the CPU and memory usage metrics, suggesting that the system is experiencing resource bottlenecks related to disk operations.

**Possible Impact**

The high disk I/O usage can lead to several possible impacts on the system, including:

1. **Performance degradation**: High disk I/O usage can lead to slower system response times, increased latency, and decreased overall performance.
2. **Resource exhaustion**: If the system is